# Análisis del riesgo de incumplimiento de los prestatarios

Se preparará un informe para la división de préstamos de un banco. Se averiguará si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

## Introducción
Se analizará los datos sobre la solvencia crediticia de los clientes de un banco para conocer y entregar a la división de préstamos del banco si un cliente tiene la capacidad o no de ser un prestatario potencial para pagar su préstamo según su estado civil, cantidad de hijos y nivel de ingresos.

### Objetivo:
1. El estado civil y el número de hijos de un cliente impacta en el incumplimiento de pago de un préstamo.
2. El nivel de ingreso influye en el pago a tiempo de un préstamo.
3. Impacto de los propósitos del préstamo al reembolso a tiempo del préstamo.

In [2]:
import pandas as pd
import numpy as np

In [3]:
credit_score = pd.read_csv('/datasets/credit_scoring_eng.csv')

## Ejercicio 1. Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

In [4]:
credit_score.shape

(21525, 12)

In [5]:
credit_score.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


Se puede evidenciar que la tabla tiene 21.525 filas y 12 columnas.
En days_employed observamos valores negativos, se llevará a valores positivos ya que no existe la experiencia laboral en negativo. Además, se debe realizar conversión de los datos de "days_employed" de días a años, o de horas a días y de días a años, según sea el caso.

In [6]:
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [7]:
credit_score.describe(percentiles=[0.25, 0.5, 0.75, 0.95, 0.99])

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
95%,2.000000,380718.999304,64.000000,1.000000,4.000000,1.000000,54494.417000
99%,3.000000,397588.993236,69.000000,3.000000,4.000000,1.000000,82753.092500
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


Las columnas days_employed y total_income tienen valores ausentes.
Los nombres de las columnas están todas en minúscula, las palabras separadas correctamente, tendríamos que asegurarnos que no tengan espacios.

In [8]:
credit_score[credit_score['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [9]:
credit_score[credit_score['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Los valores ausentes son simétricos. En el dataframe de abajo se puede evidenciar que los valores ausentes estan en las mismas filas para ambas columnas.

In [10]:
credit_score[credit_score['days_employed'].isna() & credit_score['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


In [11]:
credit_score.isna().sum() / 21525

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

**Conclusión intermedia**
El número de filas filtradas coincide con el número de valores ausentes, pudiendo decir que los valores ausentes son simétricos. Los valores ausentes de days_employed y total_income representan un 10% aprox cada uno; sin embargo, no eliminaremos ninguno de ellos para este análisis.

Se analizarán los datos para verificar si una caracteristica en especifico es la que arroja valores ausentes, y si estos valores ausentes dependen a su vez, de otros indicadores en columnas distintas de la tabla.

In [12]:
credit_score_notnan = credit_score.dropna()

In [13]:
print(credit_score_notnan["education"].value_counts(normalize=True))
print(credit_score["education"].value_counts(normalize=True))

secondary education    0.637796
bachelor's degree      0.218180
SECONDARY EDUCATION    0.036432
Secondary Education    0.033383
some college           0.031678
BACHELOR'S DEGREE      0.012971
Bachelor's Degree      0.012557
primary education      0.011937
Some College           0.002067
SOME COLLEGE           0.001137
PRIMARY EDUCATION      0.000827
Primary Education      0.000723
graduate degree        0.000207
Graduate Degree        0.000052
GRADUATE DEGREE        0.000052
Name: education, dtype: float64
secondary education    0.638792
bachelor's degree      0.219187
SECONDARY EDUCATION    0.035865
Secondary Education    0.033031
some college           0.031034
BACHELOR'S DEGREE      0.012729
Bachelor's Degree      0.012451
primary education      0.011614
Some College           0.002184
SOME COLLEGE           0.001347
PRIMARY EDUCATION      0.000790
Primary Education      0.000697
graduate degree        0.000186
GRADUATE DEGREE        0.000046
Graduate Degree        0.000046
Name: ed

In [14]:
print(credit_score_notnan["family_status"].value_counts(normalize=True))
print(credit_score["family_status"].value_counts(normalize=True))

married              0.575836
civil partnership    0.193013
unmarried            0.130484
divorced             0.055966
widow / widower      0.044701
Name: family_status, dtype: float64
married              0.575145
civil partnership    0.194053
unmarried            0.130685
divorced             0.055517
widow / widower      0.044599
Name: family_status, dtype: float64


In [15]:
print(credit_score_notnan["gender"].value_counts(normalize=True))
print(credit_score["gender"].value_counts(normalize=True))

F      0.658984
M      0.340964
XNA    0.000052
Name: gender, dtype: float64
F      0.661370
M      0.338583
XNA    0.000046
Name: gender, dtype: float64


In [16]:
print(credit_score_notnan["income_type"].value_counts(normalize=True))
print(credit_score["income_type"].value_counts(normalize=True))

employee                       0.517493
business                       0.236525
retiree                        0.177924
civil servant                  0.067800
unemployed                     0.000103
entrepreneur                   0.000052
student                        0.000052
paternity / maternity leave    0.000052
Name: income_type, dtype: float64
employee                       0.516562
business                       0.236237
retiree                        0.179141
civil servant                  0.067782
unemployed                     0.000093
entrepreneur                   0.000093
student                        0.000046
paternity / maternity leave    0.000046
Name: income_type, dtype: float64


In [17]:
print(credit_score_notnan["purpose"].value_counts(normalize=True))
print(credit_score["purpose"].value_counts(normalize=True))

wedding ceremony                            0.037259
to have a wedding                           0.035812
having a wedding                            0.035399
real estate transactions                    0.031781
buy commercial real estate                  0.030851
purchase of the house                       0.030748
buying property for renting out             0.030386
housing                                     0.030334
transactions with commercial real estate    0.030024
building a real estate                      0.029973
housing transactions                        0.029921
purchase of my own house                    0.029663
property                                    0.029559
purchase of the house for my family         0.029456
building a property                         0.028991
construction of own property                0.028939
transactions with my real estate            0.028887
buy real estate                             0.028526
buy residential real estate                 0.

Comparamos las columnas education, family_status, gender, income_type y purpose, eliminando los valores ausentes vs la data original, y se evidencio una pequeñas diferencia en la columna purpose. Posiblemente hay valores ausentes ya que las personas que solicitan el prestamos, no indican el motivo del prestamos por verguenza o por que el motivo que indiquen no le den el prestamo.

In [18]:
print(credit_score_notnan["children"].value_counts(normalize=True))
print(credit_score["children"].value_counts(normalize=True))

 0     0.656814
 1     0.224433
 2     0.095654
 3     0.015193
 20    0.003462
-1     0.002274
 4     0.001757
 5     0.000413
Name: children, dtype: float64
 0     0.657329
 1     0.223833
 2     0.095470
 3     0.015331
 20    0.003531
-1     0.002184
 4     0.001905
 5     0.000418
Name: children, dtype: float64


In [19]:
print(credit_score_notnan["family_status_id"].value_counts(normalize=True))
print(credit_score["family_status_id"].value_counts(normalize=True))

0    0.575836
1    0.193013
4    0.130484
3    0.055966
2    0.044701
Name: family_status_id, dtype: float64
0    0.575145
1    0.194053
4    0.130685
3    0.055517
2    0.044599
Name: family_status_id, dtype: float64


In [20]:
print(credit_score_notnan["total_income"].value_counts(normalize=True))
print(credit_score["total_income"].value_counts(normalize=True))

42413.096    0.000103
17312.717    0.000103
31791.384    0.000103
14427.878    0.000052
20837.034    0.000052
               ...   
27715.458    0.000052
23834.534    0.000052
26124.613    0.000052
28692.182    0.000052
41428.916    0.000052
Name: total_income, Length: 19348, dtype: float64
42413.096    0.000103
17312.717    0.000103
31791.384    0.000103
14427.878    0.000052
20837.034    0.000052
               ...   
27715.458    0.000052
23834.534    0.000052
26124.613    0.000052
28692.182    0.000052
41428.916    0.000052
Name: total_income, Length: 19348, dtype: float64


In [21]:
print(credit_score_notnan["education_id"].value_counts(normalize=True))
print(credit_score["education_id"].value_counts(normalize=True))

1    0.707612
0    0.243708
2    0.034882
3    0.013488
4    0.000310
Name: education_id, dtype: float64
1    0.707689
0    0.244367
2    0.034564
3    0.013101
4    0.000279
Name: education_id, dtype: float64


In [22]:
credit_score_notnan.describe(percentiles=[0.25, 0.5, 0.75, 0.95, 0.99])

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000,19351.000000
mean,0.537388,63046.497661,43.255336,0.819079,0.972249,0.081184,26787.568355
std,1.371408,140827.311974,12.579170,0.550104,1.420596,0.273125,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
95%,2.000000,380718.999304,64.000000,1.000000,4.000000,1.000000,54494.417000
99%,3.000000,397588.993236,69.000000,3.000000,4.000000,1.000000,82753.092500
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


**Conclusión intermedia**
Comparamos las columnas children, family_status_id, total_income y education_id, no se evidencia variación significativa respecto a valores originales y valores ausentes.

La distribución en el conjunto de datos original vs la distribución de la tabla filtrada, tiene pequeñas variaciones en la desviación estandar y la media al eliminar las filas con valores ausentes.

**Conclusiones**

Los valores ausentes de las columnas total_income y days_employed, lo primero que haremos será categorizar las edades que nos ayide a evaluar de mejor manera el ingreso total. Creando tablas donde se eliminará los valores ausentes y se calculara la media y mediana de varias columnas para escoger cual valor será el más adecuado el mejor para rellenar los valores ausentes. 

Revisaremos los valores de cada columna para verificar que todo este correcto, de no estarlo, los ajustaremos segun sea el caso para que asi podamos hacer un mejor análisis de los datos.

## Transformación de datos

In [23]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente
credit_score["education"].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [24]:
# Arregla los registros si es necesario
def replace_wrong_education(wrong_education, correct_education):
    good_education = credit_score["education"].replace(wrong_education, correct_education)
    return good_education

In [25]:
wrong_education = ["bachelor's degree", "BACHELOR'S DEGREE", "Bachelor's Degree"]
correct_education = "bachelor's_degree"

credit_score["education"] = replace_wrong_education(wrong_education, correct_education)

In [26]:
wrong_education = ["secondary education", "Secondary Education", "SECONDARY EDUCATION"]
correct_education = "secondary_education"

credit_score["education"] = replace_wrong_education(wrong_education, correct_education)

In [27]:
wrong_education = ["some college", "SOME COLLEGE", "Some College" ]
correct_education = "some_college"

credit_score["education"] = replace_wrong_education(wrong_education, correct_education)

In [28]:
wrong_education = ["primary education", "PRIMARY EDUCATION", "Primary Education"]
correct_education = "primary_education"

credit_score["education"] = replace_wrong_education(wrong_education, correct_education)

In [29]:
wrong_education = ["Graduate Degree", "GRADUATE DEGREE", "graduate degree"]
correct_education = "graduate_degree"

credit_score["education"] = replace_wrong_education(wrong_education, correct_education)

In [30]:
# Comprobar todos los valores en la columna para asegurarnos de que los hayamos corregido
credit_score["education"].unique()

array(["bachelor's_degree", 'secondary_education', 'some_college',
       'primary_education', 'graduate_degree'], dtype=object)

In [31]:
# Veamos la distribución de los valores en la columna `children`
credit_score["children"].describe()

count    21525.000000
mean         0.538908
std          1.381587
min         -1.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64

In [32]:
credit_score["children"].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

No se puede tener menos un hijo, o una persona tener 20 hijos (es posible, pero no es algo muy común), por lo tanto, pasaremos a positivo el número uno, y al número 20 lo pasaremos a dos, porque posiblemente sea error de tipeo al ingresar la data.

In [33]:
credit_score[credit_score["children"] < 0]["children"].count() / 2152

0.02184014869888476

In [34]:
credit_score[credit_score["children"] == 20]["children"].count() / 2152

0.03531598513011153

In [35]:
# [arregla los datos según tu decisión]
credit_score["children"] = credit_score["children"].replace(-1, 1)
credit_score["children"] = credit_score["children"].replace(20, 2)

In [36]:
# Comprobar la columna `children` de nuevo para asegurarnos de que todo está arreglado
credit_score["children"].unique()

array([1, 0, 3, 2, 4, 5])

In [37]:
# Encuentra datos problemáticos en `days_employed`, si existen, y calcula el porcentaje
credit_score["days_employed"].unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

Al igual que children, no puede haber valores negativos en days_employed, por lo cual aplicaremos una agregación de valor absoluto para pasar estos valores a positivos, y  tambien podemos observar que hay 401.755 días trabajados, el cual es un valor muy alto e incongruente para contar los días de experiencia laboral de una persona, entonces tomaremos como días maximos de experiencia laboral 17.155 días, y lo que sea mayor a ese número se dividira entre 24 horas para obtener un numero mas cercano a la realidad.

In [38]:
# Aborda los valores problemáticos, si existen.
credit_score["days_employed"] = abs(credit_score["days_employed"])

In [39]:
credit_score["days_employed"].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

In [40]:
credit_score.loc[credit_score["days_employed"] > 17155, "days_employed"] = credit_score.loc[credit_score["days_employed"] > 17155, "days_employed"] / 24

In [41]:
# Comprueba el resultado - asegúrate de que esté arreglado
credit_score["days_employed"].describe()

count    19351.000000
mean      4639.858099
std       5354.386044
min         24.141633
25%        926.823974
50%       2194.207271
75%       5534.630567
max      16739.808353
Name: days_employed, dtype: float64

In [42]:
credit_score.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.479721,4639.858099,43.293380,0.817236,0.972544,0.080883,26787.568355
std,0.755528,5354.386044,12.574584,0.548138,1.420324,0.272661,16475.450632
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,926.823974,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,2194.207271,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,5534.630567,53.000000,1.000000,1.000000,0.000000,32549.611000
max,5.000000,16739.808353,75.000000,4.000000,4.000000,1.000000,362496.645000


In [43]:
# Revisa `dob_years` en busca de valores sospechosos y cuenta el porcentaje
credit_score["dob_years"].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [44]:
credit_score[credit_score["dob_years"] == 0]["dob_years"].count() / 2152

0.046933085501858735

El valor problematico que tenemos es el número cero, el cual calcularemos la media de la columna dob years, y lo reempazaremos por ese valor ya que su porcentaje es de 5% aprox.

In [45]:
credit_score_median = credit_score["dob_years"].median()

In [46]:
credit_score_median

42.0

In [47]:
credit_score["dob_years"] = credit_score["dob_years"].replace(0, 42)

In [48]:
# Comprueba el resultado - asegúrate de que esté arreglado
credit_score["dob_years"].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51, 59, 29, 60, 55, 58, 71, 22, 73, 66,
       69, 19, 72, 70, 74, 75])

En la columna family_status tenemos valores tipo object con espacios, por lo cual, quitaremos los espacios a las palabras que las tengan y colocaremos el signo "_" para separas palabras.

In [49]:
# Veamos los valores de la columna
credit_score["family_status"].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

In [50]:
#credit_score["family_status"].count()
credit_score.groupby("family_status")["family_status"].count()

family_status
civil partnership     4177
divorced              1195
married              12380
unmarried             2813
widow / widower        960
Name: family_status, dtype: int64

In [51]:
def replace_wrong_family_status(wrong_family_status, correct_family_status):
    good_family_status = credit_score["family_status"].replace(wrong_family_status, correct_family_status)
    return good_family_status

In [52]:
wrong_family_status = ["civil partnership", "widow / widower"]
correct_family_status = ["civil_partnership", "widow/widower"]

credit_score["family_status"] = replace_wrong_family_status(wrong_family_status, correct_family_status)

In [53]:
# Comprueba el resultado - asegúrate de que esté arreglado
credit_score["family_status"].unique()

array(['married', 'civil_partnership', 'widow/widower', 'divorced',
       'unmarried'], dtype=object)

En la columna gender como tenemos un valor "XNA", lo reemplazaremos por el sexo M, ya que hacer esto no nos impactaria en los datos.

In [54]:
# Veamos los valores en la columna
credit_score["gender"].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [55]:
credit_score.groupby("gender")["gender"].count()

gender
F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [56]:
# Aborda los valores problemáticos, si existen
credit_score.loc[credit_score["gender"] == "XNA"]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,some_college,2,civil_partnership,1,XNA,business,0,32624.825,buy real estate


In [57]:
credit_score["gender"] = credit_score["gender"].replace("XNA", "M")

In [58]:
# Comprueba el resultado - asegúrate de que esté arreglado
credit_score["gender"].unique()

array(['F', 'M'], dtype=object)

En la columna income_type tenemos valores tipo object con espacios, el cual acomodaremos con la siguiente función.

In [59]:
# Veamos los valores en la columna
credit_score["income_type"].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

In [60]:
# Aborda los valores problemáticos, si existen
def replace_wrong_income_type(wrong_income_type, correct_income_type):
    good_income_type = credit_score["income_type"].replace(wrong_income_type, correct_income_type)
    return good_income_type

In [61]:
wrong_income_type = ["civil servant", "paternity / maternity leave"]
correct_income_type = ["civil_servant", "paternity/maternity_leave"]

credit_score["income_type"] = replace_wrong_income_type(wrong_income_type, correct_income_type)

In [62]:
# Comprueba el resultado - asegúrate de que esté arreglado
credit_score["income_type"].unique()

array(['employee', 'retiree', 'business', 'civil_servant', 'unemployed',
       'entrepreneur', 'student', 'paternity/maternity_leave'],
      dtype=object)

In [63]:
credit_score

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's_degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary_education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,secondary_education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary_education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,14177.753002,53,secondary_education,1,civil_partnership,1,F,retiree,0,25378.572,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary_education,1,civil_partnership,1,F,business,0,35966.698,housing transactions
21521,0,14330.725172,67,secondary_education,1,married,0,F,retiree,0,24959.969,purchase of a car
21522,1,2113.346888,38,secondary_education,1,civil_partnership,1,M,employee,1,14347.610,property
21523,3,3112.481705,38,secondary_education,1,married,0,M,employee,1,39054.888,buying my own car


In [64]:
# Comprobar los duplicados
credit_score.duplicated().sum()

72

In [65]:
duplicated_credit_score = credit_score[credit_score.duplicated()]
duplicated_credit_score

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,secondary_education,1,married,0,F,employee,0,NaN,purchase of the house for my family
3290,0,NaN,58,secondary_education,1,civil_partnership,1,F,retiree,0,NaN,to have a wedding
4182,1,NaN,34,bachelor's_degree,0,civil_partnership,1,F,employee,0,NaN,wedding ceremony
4851,0,NaN,60,secondary_education,1,civil_partnership,1,F,retiree,0,NaN,wedding ceremony
5557,0,NaN,58,secondary_education,1,civil_partnership,1,F,retiree,0,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64,secondary_education,1,married,0,F,retiree,0,NaN,supplementary education
21032,0,NaN,60,secondary_education,1,married,0,F,retiree,0,NaN,to become educated
21132,0,NaN,47,secondary_education,1,married,0,F,employee,0,NaN,housing renovation
21281,1,NaN,30,bachelor's_degree,0,married,0,F,employee,0,NaN,buy commercial real estate


In [66]:
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Podemos ver que tenemos 72 filas duplicadas, las cuales no eliminaremos porque en estos momentos no tenemos un ID que nos permita realmente decir que son duplicados, por lo cual, reemplazaremos los valores ausentes.

# Trabajar con valores ausentes

Aunque no sabemos si utilizaremos diccionarios, podemos crearemos dos, que son education y family status al tener education_id y family_status_id.

In [67]:
# Encuentra los diccionarios
credit_score.set_index("education_id")["education"].to_dict()

{0: "bachelor's_degree",
 1: 'secondary_education',
 2: 'some_college',
 3: 'primary_education',
 4: 'graduate_degree'}

In [68]:
credit_score.set_index("family_status_id")["family_status"].to_dict()

{0: 'married',
 1: 'civil_partnership',
 2: 'widow/widower',
 3: 'divorced',
 4: 'unmarried'}

### Restaurar valores ausentes en `total_income`

Como se pudo evidenciar al inicio del proyecto, las columnas total income y days employes tienen valores ausentes. Crearemos una nueva columna con categorias de edad, esto nos permitirá hacer un mejor análisis y manejo de la data. Calcularemos la media y la mediana excluyendo las filas con valores ausentes, y escogeremos los factores que mas se relacionen mejor con ambas columnas, para sustituir los valores ausentes.

In [69]:
credit_score["dob_years"].min()

19

In [70]:
credit_score["dob_years"].max()

75

In [71]:
# Vamos a escribir una función que calcule la categoría de edad
def age_category(age):
    if age < 30:
        return "19-29"
    elif age < 40:
        return "30-39"
    elif age < 50:
        return "40-49"
    elif age < 60:
        return "50-59"
    elif age < 70:
        return "60-69"
    else:
        return "70+"

In [72]:
print(age_category(25))
print(age_category(38))
print(age_category(43))
print(age_category(59))
print(age_category(61))
print(age_category(74))

19-29
30-39
40-49
50-59
60-69
70+


In [73]:
# Crear una nueva columna basada en la función
credit_score["dob_years_category"] = credit_score["dob_years"].apply(age_category)

In [74]:
# Comprobar cómo los valores en la nueva columna
credit_score

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_category
0,1,8437.673028,42,bachelor's_degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36,secondary_education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33,secondary_education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32,secondary_education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,14177.753002,53,secondary_education,1,civil_partnership,1,F,retiree,0,25378.572,to have a wedding,50-59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary_education,1,civil_partnership,1,F,business,0,35966.698,housing transactions,40-49
21521,0,14330.725172,67,secondary_education,1,married,0,F,retiree,0,24959.969,purchase of a car,60-69
21522,1,2113.346888,38,secondary_education,1,civil_partnership,1,M,employee,1,14347.610,property,30-39
21523,3,3112.481705,38,secondary_education,1,married,0,M,employee,1,39054.888,buying my own car,30-39


In [75]:
# Crea una tabla sin valores ausentes y muestra algunas de sus filas para asegurarte de que se ve bien
credit_score_notnan = credit_score.dropna()
credit_score_notnan.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_category
0,1,8437.673028,42,bachelor's_degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36,secondary_education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33,secondary_education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32,secondary_education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,14177.753002,53,secondary_education,1,civil_partnership,1,F,retiree,0,25378.572,to have a wedding,50-59


In [76]:
# Examina los valores medios de los ingresos en función de los factores que identificaste, mean
credit_score_notnan.pivot_table(index=["dob_years_category", "income_type"], values="total_income", aggfunc="mean")

total_income
dob_years_category income_type                            
19-29              business                   28690.055945
                   civil_servant              25145.840248
                   employee                   24099.859203
                   entrepreneur               79866.103000
                   retiree                    14888.651857
                   student                    15712.260000
30-39              business                   33145.949183
                   civil_servant              27921.836553
                   employee                   26191.716908
                   paternity/maternity_leave   8612.661000
                   retiree                    23122.709862
                   unemployed                  9593.119000
40-49              business                   33932.535312
                   civil_servant              28493.419941
                   employee                   26175.155197
                   retiree                    26446.228569
                   unemployed                 32435.602000
50-59              business                   32385.032725
                   civil_servant              25838.105730
                   employee                   26073.759931
                   retiree                    22221.765833
60-69              business                   32494.918350
                   civil_servant              29305.166039
                   employee                   27307.606610
                   retiree                    21544.426743
70+                business                   27766.307200
                   civil_servant              32189.795667
                   employee                   26672.382429
                   retiree                    18994.044264

In [77]:
# Examina los valores medianos de los ingresos en función de los factores que identificaste, medium
credit_score_notnan.pivot_table(index=["dob_years_category", "income_type"], values="total_income", aggfunc="median")

total_income
dob_years_category income_type                            
19-29              business                     25570.8550
                   civil_servant                23504.5105
                   employee                     21537.4790
                   entrepreneur                 79866.1030
                   retiree                      12807.0710
                   student                      15712.2600
30-39              business                     28765.5935
                   civil_servant                24662.7440
                   employee                     23218.8030
                   paternity/maternity_leave     8612.6610
                   retiree                      18735.7160
                   unemployed                    9593.1190
40-49              business                     28700.6110
                   civil_servant                24828.9160
                   employee                     23108.1500
                   retiree                      22262.3090
                   unemployed                   32435.6020
50-59              business                     27299.6345
                   civil_servant                23136.3020
                   employee                     22547.8310
                   retiree                      19526.8120
60-69              business                     29171.9890
                   civil_servant                23390.0570
                   employee                     23316.9650
                   retiree                      18446.1435
70+                business                     28138.8950
                   civil_servant                24525.2240
                   employee                     24660.9010
                   retiree                      17650.4660

In [78]:
credit_score_notnan.pivot_table(index=["dob_years_category", "income_type"], values="total_income", aggfunc="mean") - credit_score_notnan.pivot_table(index=["dob_years_category", "income_type"], values="total_income", aggfunc="median")

total_income
dob_years_category income_type                            
19-29              business                    3119.200945
                   civil_servant               1641.329748
                   employee                    2562.380203
                   entrepreneur                   0.000000
                   retiree                     2081.580857
                   student                        0.000000
30-39              business                    4380.355683
                   civil_servant               3259.092553
                   employee                    2972.913908
                   paternity/maternity_leave      0.000000
                   retiree                     4386.993862
                   unemployed                     0.000000
40-49              business                    5231.924312
                   civil_servant               3664.503941
                   employee                    3067.005197
                   retiree                     4183.919569
                   unemployed                     0.000000
50-59              business                    5085.398225
                   civil_servant               2701.803730
                   employee                    3525.928931
                   retiree                     2694.953833
60-69              business                    3322.929350
                   civil_servant               5915.109039
                   employee                    3990.641610
                   retiree                     3098.283243
70+                business                    -372.587800
                   civil_servant               7664.571667
                   employee                    2011.481429
                   retiree                     1343.578264

In [79]:
t = credit_score_notnan.pivot_table(index=["dob_years_category", "income_type"], values="total_income", aggfunc="median")

In [80]:
t["total_income"]["19-29"]["civil_servant"]

23504.5105

In [81]:
credit_score_notnan.pivot_table(index=["dob_years_category", "gender"], values="total_income", aggfunc="mean")

total_income
dob_years_category gender              
19-29              F       23142.354408
                   M       28768.286201
30-39              F       25701.974072
                   M       32410.764112
40-49              F       25941.849590
                   M       33277.796565
50-59              F       24361.606104
                   M       29634.446544
60-69              F       22587.891752
                   M       25399.489363
70+                F       19203.044387
                   M       23303.550806

In [82]:
credit_score_notnan.pivot_table(index=["dob_years_category", "gender"], values="total_income", aggfunc="median")

total_income
dob_years_category gender              
19-29              F         20431.2345
                   M         26002.8340
30-39              F         22328.9650
                   M         28161.4955
40-49              F         22672.9320
                   M         28401.6975
50-59              F         21028.0100
                   M         25703.8170
60-69              F         19431.7180
                   M         21472.5740
70+                F         17864.7065
                   M         20526.6140

In [83]:
credit_score_notnan.pivot_table(index=["dob_years_category", "gender"], values="total_income", aggfunc="mean") - credit_score_notnan.pivot_table(index=["dob_years_category", "gender"], values="total_income", aggfunc="median")

total_income
dob_years_category gender              
19-29              F        2711.119908
                   M        2765.452201
30-39              F        3373.009072
                   M        4249.268612
40-49              F        3268.917590
                   M        4876.099065
50-59              F        3333.596104
                   M        3930.629544
60-69              F        3156.173752
                   M        3926.915363
70+                F        1338.337887
                   M        2776.936806

In [84]:
credit_score_notnan.pivot_table(index=["dob_years_category", "education"], values="total_income", aggfunc="mean")

total_income
dob_years_category education                        
19-29              bachelor's_degree    29395.106109
                   primary_education    27695.271520
                   secondary_education  23379.052855
                   some_college         25292.291928
30-39              bachelor's_degree    34225.243752
                   graduate_degree      18187.301500
                   primary_education    21979.584515
                   secondary_education  25666.783012
                   some_college         31991.246531
40-49              bachelor's_degree    35497.434965
                   graduate_degree      31771.321000
                   primary_education    23618.267667
                   secondary_education  26146.581638
                   some_college         33648.706685
50-59              bachelor's_degree    33622.674858
                   graduate_degree      42945.794000
                   primary_education    17797.322623
                   secondary_education  24057.271660
                   some_college         27623.492353
60-69              bachelor's_degree    30329.905325
                   graduate_degree      28334.215000
                   primary_education    18710.592883
                   secondary_education  21691.515498
                   some_college         30476.594957
70+                bachelor's_degree    26173.068696
                   primary_education    18892.886000
                   secondary_education  19245.043953
                   some_college         13917.989667

In [85]:
credit_score_notnan.pivot_table(index=["dob_years_category", "education"], values="total_income", aggfunc="median")

total_income
dob_years_category education                        
19-29              bachelor's_degree      25956.1640
                   primary_education      25488.9160
                   secondary_education    21114.7620
                   some_college           22687.1980
30-39              bachelor's_degree      28794.9310
                   graduate_degree        18187.3015
                   primary_education      19542.3265
                   secondary_education    22912.9930
                   some_college           28463.4390
40-49              bachelor's_degree      30094.2090
                   graduate_degree        31771.3210
                   primary_education      21511.5635
                   secondary_education    22960.5920
                   some_college           29323.6730
50-59              bachelor's_degree      28152.1765
                   graduate_degree        42945.7940
                   primary_education      16922.6250
                   secondary_education    21245.4820
                   some_college           21132.3090
60-69              bachelor's_degree      25222.3445
                   graduate_degree        28334.2150
                   primary_education      17657.4995
                   secondary_education    18873.7640
                   some_college           28178.9170
70+                bachelor's_degree      25497.3920
                   primary_education      15013.5050
                   secondary_education    18508.5770
                   some_college           14479.1930

In [86]:
credit_score_notnan.pivot_table(index=["dob_years_category", "education"], values="total_income", aggfunc="mean") - credit_score_notnan.pivot_table(index=["dob_years_category", "education"], values="total_income", aggfunc="median")

total_income
dob_years_category education                        
19-29              bachelor's_degree     3438.942109
                   primary_education     2206.355520
                   secondary_education   2264.290855
                   some_college          2605.093928
30-39              bachelor's_degree     5430.312752
                   graduate_degree          0.000000
                   primary_education     2437.258015
                   secondary_education   2753.790012
                   some_college          3527.807531
40-49              bachelor's_degree     5403.225965
                   graduate_degree          0.000000
                   primary_education     2106.704167
                   secondary_education   3185.989638
                   some_college          4325.033685
50-59              bachelor's_degree     5470.498358
                   graduate_degree          0.000000
                   primary_education      874.697623
                   secondary_education   2811.789660
                   some_college          6491.183353
60-69              bachelor's_degree     5107.560825
                   graduate_degree          0.000000
                   primary_education     1053.093383
                   secondary_education   2817.751498
                   some_college          2297.677957
70+                bachelor's_degree      675.676696
                   primary_education     3879.381000
                   secondary_education    736.466953
                   some_college          -561.203333

Comparamos la media y la mediana con income_type, gender y education para verificar con cual factor podriamos definir mejor los ingresos. Se escogió la mediana  porque evidencio tener valores atipicos, y se calculó con base en la categoria de income type.

In [87]:
#  Escribe una función que usaremos para completar los valores ausentes
def income(row):
    total_income = row["total_income"]
    dob_years_category = row["dob_years_category"]
    income_type = row["income_type"]
    if pd.isna(total_income):
        try:
            return t["total_income"][dob_years_category][income_type]
        except:
            return None
    return total_income

In [88]:
# Comprueba si funciona
credit_score.apply(income, axis=1).isna().sum()

1

In [89]:
# Aplícalo a cada fila
credit_score["total_income"] = credit_score.apply(income, axis=1)
credit_score.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_category
0,1,8437.673028,42,bachelor's_degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36,secondary_education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33,secondary_education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32,secondary_education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,14177.753002,53,secondary_education,1,civil_partnership,1,F,retiree,0,25378.572,to have a wedding,50-59
5,0,926.185831,27,bachelor's_degree,0,civil_partnership,1,M,business,0,40922.170,purchase of the house,19-29
6,0,2879.202052,43,bachelor's_degree,0,married,0,F,business,0,38484.156,housing transactions,40-49
7,0,152.779569,50,secondary_education,1,married,0,M,employee,0,21731.829,education,50-59
8,2,6929.865299,35,bachelor's_degree,0,civil_partnership,1,F,employee,0,15337.093,having a wedding,30-39
9,0,2188.756445,41,secondary_education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-49


In [90]:
# Comprueba si tenemos algún error
credit_score.loc[credit_score["total_income"].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_category
5936,0,NaN,58,bachelor's_degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,50-59


**Reemplazar los valores ausentes si hay algún error**

In [92]:
# Comprobar el número de entradas en las columnas
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   children            21525 non-null  int64  
 1   days_employed       19351 non-null  float64
 2   dob_years           21525 non-null  int64  
 3   education           21525 non-null  object 
 4   education_id        21525 non-null  int64  
 5   family_status       21525 non-null  object 
 6   family_status_id    21525 non-null  int64  
 7   gender              21525 non-null  object 
 8   income_type         21525 non-null  object 
 9   debt                21525 non-null  int64  
 10  total_income        21524 non-null  float64
 11  purpose             21525 non-null  object 
 12  dob_years_category  21525 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


###  Restaurar valores en `days_employed`

In [93]:
# Distribución de las medianas de `days_employed` en función de los parámetros identificados
credit_score_notnan.pivot_table(index=["dob_years_category", "education"], values="days_employed", aggfunc="median")

days_employed
dob_years_category education                         
19-29              bachelor's_degree      1011.683940
                   primary_education      1030.219648
                   secondary_education    1016.142203
                   some_college            885.268574
30-39              bachelor's_degree      1598.171398
                   graduate_degree        4159.753909
                   primary_education      1110.847169
                   secondary_education    1641.266192
                   some_college           1209.230373
40-49              bachelor's_degree      2078.916432
                   graduate_degree         409.200149
                   primary_education      1108.837429
                   secondary_education    2129.430666
                   some_college           2023.682734
50-59              bachelor's_degree      4099.473961
                   graduate_degree        5352.038180
                   primary_education     13936.812211
                   secondary_education    5116.895774
                   some_college           2917.701025
60-69              bachelor's_degree     14274.756778
                   graduate_degree       14872.104898
                   primary_education     14922.497849
                   secondary_education   14847.817475
                   some_college          14539.637146
70+                bachelor's_degree     14693.128559
                   primary_education     15026.662798
                   secondary_education   15084.753322
                   some_college          16517.600603

In [94]:
# Distribución de las medias de `days_employed` en función de los parámetros identificados
credit_score_notnan.pivot_table(index=["dob_years_category", "education"], values="days_employed", aggfunc="mean")

days_employed
dob_years_category education                         
19-29              bachelor's_degree      1242.000024
                   primary_education      1236.562385
                   secondary_education    1277.581057
                   some_college           1045.216936
30-39              bachelor's_degree      2070.263458
                   graduate_degree        4159.753909
                   primary_education      1917.349001
                   secondary_education    2154.055571
                   some_college           1574.576678
40-49              bachelor's_degree      3028.074601
                   graduate_degree         409.200149
                   primary_education      2385.643500
                   secondary_education    3136.014169
                   some_college           2762.906793
50-59              bachelor's_degree      6779.372399
                   graduate_degree        5352.038180
                   primary_education      9820.016842
                   secondary_education    7703.461939
                   some_college           6422.634711
60-69              bachelor's_degree     11270.864787
                   graduate_degree       14872.104898
                   primary_education     13807.820963
                   secondary_education   12936.974430
                   some_college          10481.773399
70+                bachelor's_degree     14277.393474
                   primary_education     15023.664625
                   secondary_education   13689.173039
                   some_college          15912.501887

In [95]:
Y = credit_score_notnan.pivot_table(index=["dob_years_category", "education"], values="days_employed", aggfunc="median")

In [96]:
Y["days_employed"]["19-29"]["secondary_education"]

1016.142203152586

In [97]:
credit_score_notnan.pivot_table(index=["dob_years_category", "education"], values="days_employed", aggfunc="median") - credit_score_notnan.pivot_table(index=["dob_years_category", "education"], values="days_employed", aggfunc="mean")

days_employed
dob_years_category education                         
19-29              bachelor's_degree      -230.316084
                   primary_education      -206.342737
                   secondary_education    -261.438854
                   some_college           -159.948362
30-39              bachelor's_degree      -472.092060
                   graduate_degree           0.000000
                   primary_education      -806.501833
                   secondary_education    -512.789379
                   some_college           -365.346305
40-49              bachelor's_degree      -949.158169
                   graduate_degree           0.000000
                   primary_education     -1276.806071
                   secondary_education   -1006.583502
                   some_college           -739.224059
50-59              bachelor's_degree     -2679.898439
                   graduate_degree           0.000000
                   primary_education      4116.795368
                   secondary_education   -2586.566165
                   some_college          -3504.933686
60-69              bachelor's_degree      3003.891990
                   graduate_degree           0.000000
                   primary_education      1114.676885
                   secondary_education    1910.843045
                   some_college           4057.863746
70+                bachelor's_degree       415.735085
                   primary_education         2.998174
                   secondary_education    1395.580284
                   some_college            605.098715

Comparamos la media y la mediana con education. Se escogió la mediana porque evidencio tener valores atipicos.

In [98]:
# Escribamos una función que calcule medias o medianas (dependiendo de tu decisión) según el parámetro identificado
def employed(row):
    days_employed = row["days_employed"]
    dob_years_category = row["dob_years_category"]
    education = row["education"]
    if pd.isna(days_employed):
        try:
            return Y["days_employed"][dob_years_category][education]
        except:
            return None
    return days_employed

In [99]:
employed

<function __main__.employed(row)>

In [709]:
# Comprueba que la función funciona
credit_score.apply(employed, axis=1).isna().sum()

0

In [710]:
# Aplicar la función al income_type
credit_score["days_employed"] = credit_score.apply(employed, axis=1)
credit_score.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_category
0,1,8437.673028,42,bachelor's_degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36,secondary_education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33,secondary_education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32,secondary_education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,14177.753002,53,secondary_education,1,civil_partnership,1,F,retiree,0,25378.572,to have a wedding,50-59
5,0,926.185831,27,bachelor's_degree,0,civil_partnership,1,M,business,0,40922.170,purchase of the house,19-29
6,0,2879.202052,43,bachelor's_degree,0,married,0,F,business,0,38484.156,housing transactions,40-49
7,0,152.779569,50,secondary_education,1,married,0,M,employee,0,21731.829,education,50-59
8,2,6929.865299,35,bachelor's_degree,0,civil_partnership,1,F,employee,0,15337.093,having a wedding,30-39
9,0,2188.756445,41,secondary_education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,40-49


In [711]:
# Comprueba si la función funcionó
credit_score.loc[credit_score["days_employed"].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_category


In [712]:
# Comprueba las entradas en todas las columnas: asegúrate de que hayamos corregido todos los valores ausentes
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   children            21525 non-null  int64  
 1   days_employed       21525 non-null  float64
 2   dob_years           21525 non-null  int64  
 3   education           21525 non-null  object 
 4   education_id        21525 non-null  int64  
 5   family_status       21525 non-null  object 
 6   family_status_id    21525 non-null  int64  
 7   gender              21525 non-null  object 
 8   income_type         21525 non-null  object 
 9   debt                21525 non-null  int64  
 10  total_income        21524 non-null  float64
 11  purpose             21525 non-null  object 
 12  dob_years_category  21525 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 2.1+ MB


## Clasificación de datos

In [713]:
# Muestra los valores de los datos seleccionados para la clasificación
credit_score["children"].value_counts() #OK

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [714]:
credit_score["debt"].value_counts() #OK

0    19784
1     1741
Name: debt, dtype: int64

In [715]:
credit_score["purpose"].value_counts().sort_values(ascending=False) #OK

wedding ceremony                            797
having a wedding                            777
to have a wedding                           774
real estate transactions                    676
buy commercial real estate                  664
buying property for renting out             653
housing transactions                        653
transactions with commercial real estate    651
purchase of the house                       647
housing                                     647
purchase of the house for my family         641
construction of own property                635
property                                    634
transactions with my real estate            630
building a real estate                      626
buy real estate                             624
building a property                         620
purchase of my own house                    620
housing renovation                          612
buy residential real estate                 607
buying my own car                       

In [716]:
credit_score["total_income"].value_counts()

23108.1500    348
23218.8030    345
21537.4790    195
19526.8120    191
18446.1435    190
             ... 
10000.3920      1
99284.6960      1
6264.5320       1
27097.0850      1
41428.9160      1
Name: total_income, Length: 19358, dtype: int64

In [717]:
# Comprobar los valores únicos

In [718]:
credit_score["children"].unique() #OK

array([1, 0, 3, 2, 4, 5])

In [719]:
credit_score["debt"].unique() #OK

array([0, 1])

In [720]:
credit_score["purpose"].unique() #OK

array(['purchase of the house', 'car purchase', 'supplementary education',
       'to have a wedding', 'housing transactions', 'education',
       'having a wedding', 'purchase of the house for my family',
       'buy real estate', 'buy commercial real estate',
       'buy residential real estate', 'construction of own property',
       'property', 'building a property', 'buying a second-hand car',
       'buying my own car', 'transactions with commercial real estate',
       'building a real estate', 'housing',
       'transactions with my real estate', 'cars', 'to become educated',
       'second-hand car purchase', 'getting an education', 'car',
       'wedding ceremony', 'to get a supplementary education',
       'purchase of my own house', 'real estate transactions',
       'getting higher education', 'to own a car', 'purchase of a car',
       'profile education', 'university education',
       'buying property for renting out', 'to buy a car',
       'housing renovation', 'going

In [721]:
credit_score["total_income"].unique()

array([40620.102, 17932.802, 23341.752, ..., 14347.61 , 39054.888,
       13127.587])

Tenemos dos grupos principales: purpose y total_income, el cual se clasificara.

In [722]:
# Escribamos una función para clasificar los datos en función de temas comunes
def purpose_category(purpose):
    if "car" in purpose:
        return "car_purpose"
    if "educat" in purpose or "university" in purpose:
        return "education_purpose"
    if "hous" in purpose or "property" in purpose or "real estate" in purpose:
        return "property_purpose"
    return "wedding_purpose"

In [723]:
print(purpose_category("buying my own car"))
print(purpose_category("getting an education"))
print(purpose_category("housing"))
print(purpose_category("property"))
print(purpose_category("building a real estate"))
print(purpose_category("university education"))
print(purpose_category("to have a wedding"))

car_purpose
education_purpose
property_purpose
property_purpose
property_purpose
education_purpose
wedding_purpose


In [724]:
credit_score["purpose_category"] = credit_score["purpose"].apply(purpose_category)

In [725]:
credit_score

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_category,purpose_category
0,1,8437.673028,42,bachelor's_degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49,property_purpose
1,1,4024.803754,36,secondary_education,1,married,0,F,employee,0,17932.802,car purchase,30-39,car_purpose
2,0,5623.422610,33,secondary_education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39,property_purpose
3,3,4124.747207,32,secondary_education,1,married,0,M,employee,0,42820.568,supplementary education,30-39,education_purpose
4,0,14177.753002,53,secondary_education,1,civil_partnership,1,F,retiree,0,25378.572,to have a wedding,50-59,wedding_purpose
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary_education,1,civil_partnership,1,F,business,0,35966.698,housing transactions,40-49,property_purpose
21521,0,14330.725172,67,secondary_education,1,married,0,F,retiree,0,24959.969,purchase of a car,60-69,car_purpose
21522,1,2113.346888,38,secondary_education,1,civil_partnership,1,M,employee,1,14347.610,property,30-39,property_purpose
21523,3,3112.481705,38,secondary_education,1,married,0,M,employee,1,39054.888,buying my own car,30-39,car_purpose


In [726]:
# Crea una columna con las categorías y cuenta los valores en ellas
credit_score["purpose_category"].value_counts()

property_purpose     10840
car_purpose           4315
education_purpose     4022
wedding_purpose       2348
Name: purpose_category, dtype: int64

In [727]:
# Revisar todos los datos numéricos en la columna seleccionada para la clasificación
def not_children(children):
    if children > 0:
        return "yes_children"
    else:
        return "not_children"

In [728]:
print(not_children(0))
print(not_children(4))

not_children
yes_children


In [729]:
def payment_on_time(payment):
    if payment == 0:
        return "not_debt"
    else:
        return "yes_debt"

In [730]:
print(payment_on_time(0))
print(payment_on_time(1))

not_debt
yes_debt


In [731]:
credit_score["children?"] = credit_score["children"].apply(not_children)

In [732]:
credit_score["payment_on_time"] = credit_score["debt"].apply(payment_on_time)

In [733]:
credit_score

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_category,purpose_category,children?,payment_on_time
0,1,8437.673028,42,bachelor's_degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49,property_purpose,yes_children,not_debt
1,1,4024.803754,36,secondary_education,1,married,0,F,employee,0,17932.802,car purchase,30-39,car_purpose,yes_children,not_debt
2,0,5623.422610,33,secondary_education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39,property_purpose,not_children,not_debt
3,3,4124.747207,32,secondary_education,1,married,0,M,employee,0,42820.568,supplementary education,30-39,education_purpose,yes_children,not_debt
4,0,14177.753002,53,secondary_education,1,civil_partnership,1,F,retiree,0,25378.572,to have a wedding,50-59,wedding_purpose,not_children,not_debt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary_education,1,civil_partnership,1,F,business,0,35966.698,housing transactions,40-49,property_purpose,yes_children,not_debt
21521,0,14330.725172,67,secondary_education,1,married,0,F,retiree,0,24959.969,purchase of a car,60-69,car_purpose,not_children,not_debt
21522,1,2113.346888,38,secondary_education,1,civil_partnership,1,M,employee,1,14347.610,property,30-39,property_purpose,yes_children,yes_debt
21523,3,3112.481705,38,secondary_education,1,married,0,M,employee,1,39054.888,buying my own car,30-39,car_purpose,yes_children,yes_debt


In [734]:
# Obtener estadísticas resumidas para la columna
credit_score["total_income"].describe(percentiles=[0.25, 0.5, 0.75, 0.95, 0.99])

count     21524.000000
mean      26443.608161
std       15685.309060
min        3306.762000
25%       17245.929000
50%       23132.256000
75%       31287.232000
95%       53020.827600
99%       80812.578580
max      362496.645000
Name: total_income, dtype: float64

In [736]:
# Crear una función para clasificar en diferentes grupos numéricos basándose en rangos
def total_income_category(income):
    if income < 17000:
        return "very_low"
    elif income < 23000:
        return "low"
    elif income < 53000:
        return "high"
    else:
        return "very_high"

In [737]:
print(total_income_category(16500))
print(total_income_category(21500))
print(total_income_category(50500))
print(total_income_category(80500))

very_low
low
high
very_high


In [738]:
credit_score["total_income_category"] = credit_score["total_income"].apply(total_income_category)

In [739]:
credit_score

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_category,purpose_category,children?,payment_on_time,total_income_category
0,1,8437.673028,42,bachelor's_degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49,property_purpose,yes_children,not_debt,high
1,1,4024.803754,36,secondary_education,1,married,0,F,employee,0,17932.802,car purchase,30-39,car_purpose,yes_children,not_debt,low
2,0,5623.422610,33,secondary_education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39,property_purpose,not_children,not_debt,high
3,3,4124.747207,32,secondary_education,1,married,0,M,employee,0,42820.568,supplementary education,30-39,education_purpose,yes_children,not_debt,high
4,0,14177.753002,53,secondary_education,1,civil_partnership,1,F,retiree,0,25378.572,to have a wedding,50-59,wedding_purpose,not_children,not_debt,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary_education,1,civil_partnership,1,F,business,0,35966.698,housing transactions,40-49,property_purpose,yes_children,not_debt,high
21521,0,14330.725172,67,secondary_education,1,married,0,F,retiree,0,24959.969,purchase of a car,60-69,car_purpose,not_children,not_debt,high
21522,1,2113.346888,38,secondary_education,1,civil_partnership,1,M,employee,1,14347.610,property,30-39,property_purpose,yes_children,yes_debt,very_low
21523,3,3112.481705,38,secondary_education,1,married,0,M,employee,1,39054.888,buying my own car,30-39,car_purpose,yes_children,yes_debt,high


In [740]:
# Contar los valores de cada categoría para ver la distribución
credit_score.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21524.000000
mean,0.479721,4576.549769,43.490453,0.817236,0.972544,0.080883,26443.608161
std,0.755528,5257.094554,12.218595,0.548138,1.420324,0.272661,15685.309060
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,1011.683940,34.000000,1.000000,0.000000,0.000000,17245.929000
50%,0.000000,2129.430666,42.000000,1.000000,0.000000,0.000000,23132.256000
75%,1.000000,5125.978707,53.000000,1.000000,1.000000,0.000000,31287.232000
max,5.000000,16739.808353,75.000000,4.000000,4.000000,1.000000,362496.645000


## Comprobación de las hipótesis


In [741]:
credit_score['not_debt'] = np.where(credit_score['payment_on_time'] == 'not_debt', 1, 0)

In [742]:
credit_score['yes_debt'] = np.where(credit_score['payment_on_time'] == 'yes_debt', 1, 0)

In [743]:
credit_score[['payment_on_time', 'not_debt']].head(15)

,payment_on_time,not_debt
0,not_debt,1
1,not_debt,1
2,not_debt,1
3,not_debt,1
4,not_debt,1
5,not_debt,1
6,not_debt,1
7,not_debt,1
8,not_debt,1
9,not_debt,1


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [744]:
# Calculando la tasa de incumplimiento en función del número de hijos
credit_score.pivot_table(index='children', values='not_debt', aggfunc='mean')

,not_debt
children,
0,0.924871
1,0.908530
2,0.905209
3,0.918182
4,0.902439
5,1.000000


**Conclusión**

El tener mayor cantidad de hijos, la persona que solicitó el prestamo más paga a tiempo.

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [745]:
# Comprueba los datos del estado familiar y los pagos a tiempo
# Calcular la tasa de incumplimiento basada en el estado familiar
credit_score.pivot_table(index='family_status', values='not_debt', aggfunc='mean')

,not_debt
family_status,
civil_partnership,0.907110
divorced,0.928870
married,0.924798
unmarried,0.902595
widow/widower,0.934375


**Conclusión**

El pago a tiempo con base en family status, se evidencia que los viudos y los divorciados tienen un 93% en responsabilidad de pago, 92% los casados, 91% sociedad civil y por último con un 90% los solteros. Donde las personas solteras tienen el porcentaje menor respecto al pago a tiempo de prestamos.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [746]:
# Comprueba los datos del nivel de ingresos y los pagos a tiempo
# Calcular la tasa de incumplimiento basada en el nivel de ingresos
credit_score.pivot_table(index='total_income_category', values='not_debt', aggfunc='mean')

,not_debt
total_income_category,
high,0.920824
low,0.911094
very_high,0.928770
very_low,0.921617


In [747]:
credit_score["total_income_category"].value_counts()

high         10142
very_low      5218
low           5084
very_high     1081
Name: total_income_category, dtype: int64

**Conclusión**

Al ver la relación entre el nivel de ingreso y el pago a tiempo, se puede observar que no hay mucha diferencia entre las categorias de ingreso.

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [748]:
# Consulta los porcentajes de tasa de incumplimiento para cada propósito del crédito y analízalos
credit_score.pivot_table(index='purpose_category', values='yes_debt', aggfunc='mean')

,yes_debt
purpose_category,
car_purpose,0.093395
education_purpose,0.091994
property_purpose,0.072140
wedding_purpose,0.079216


**Conclusión**

Se evidencia que cuando solicitan prestamos relacionados con un auto el porcentaje de incumplimiento comparado con las otras categorias es mayor, con un valor de 9.3%, seguida por la categoria de educación. Y la categoria más baja fue todo lo relacionado a propiedades con un 7.2% 

# Conclusión general 

Se ha evidenciado las siguientes hipótesis:

1.- El nivel de ingreso de las personas que solicitan un préstamos no tiene mucho impacto al momento de pagar la deuda.
2.- Las personas solteras tienen un menor compromiso de pagar las deudas, respecto a las personas divorciadas o viudas.
3.- Las personas al tener cinco hijos, cancelan más sus deudas. Las personas que tienen de 0 a 4 hijos, están entre un rango de 90%-92% de pagar sus préstamos.
4.- La categoría con mayor porcentaje de deuda fue la que tiene el propósito relacionado con los autos, estando la educación en un segundo lugar, y matrimonio y propiedad en tercer lugar.

Para obtener estos resultados, tuvimos que identificar si en la data teníamos valores ausentes, donde en este proyecto fueron las columnas days_employed y total_income. Transformamos los datos en la columna education, family status e income type. En la columna children encontramos valores negativos y valores muy grandes el cual ambos fueron ajustados.Days employed transformamos los valores mayor a 17155 días, ya que al ingresar la info por el valor de los montos pudieron haber ingresado horas y no días. Dob years se calculo la mediana para poder reemplazar los valores ceros y en gender, se vislumbro un valor "XNA" el cual al ser un solo valor, y no generaria distorsión de modificarse, lo agregamos al sexo masculino.

No eliminamos los valores ausentes, por no contar con un ID que nos permitiera ver si realmente esos valores eran duplicados, o no tenía relación con un patrón de comportamiento en los datos tomados por el banco.

Trabajamos con los valores ausentes en las columnas total income y days employed, haciendo categorización en dob years y en purpose ya que nos ayudaba a calcular mejor los valores que tuvieran relación con estas dos columnas. Calculamos la media y la mediana, en distintos factores y así decidir cual seria el correcto para restaurar los valores ausentes, el cual es este caso se escogió la mediana en relación a ingresos y tipo de empleo para total income, e ingresos y educación para days employed por reflejar valores atípicos.